In [ ]:
%pip install requests

In [ ]:
%pip install pandas

In [ ]:
%pip install tqdm

In [ ]:
import pandas as pd
import numpy as np
import requests
from tqdm.auto import tqdm

In [ ]:
df = pd.read_csv("merged.csv")
df

In [ ]:
df = df[df["ULICA"].notna()]

In [ ]:
# get the first number from NUMERY
df['street_number'] = df['NUMERY'].str.split('[-,]').str[0]

In [ ]:
URL = "http://localhost:8080"

In [ ]:
def get_region(city, street, number) -> tuple:
    if pd.isna(number):
        number = "1"
    res = requests.get(URL + "/search", params={"q": " ".join((city, street, number))})
    props = res.json()
    if not len(props):
        return (None, None, None)

    place = props[0]
    
    res = requests.get(URL + "/details", params={
        "place_id": place["place_id"],
        "addressdetails": 1
    })

    address_details = res.json()["address"]

    boundaries = []

    for el in address_details:
        adm_lvl = el.get("admin_level")
        if adm_lvl is None or el["class"] != "boundary" or adm_lvl < 9:
            continue

        boundaries.append(el)

    boundaries_lowest_lvl_nearest = [*sorted(boundaries, key=lambda x: x["distance"])]
    if not len(boundaries_lowest_lvl_nearest):
        return (None, place["lon"], place["lat"])

    return (boundaries_lowest_lvl_nearest[0]["localname"], place["lon"], place["lat"])
    

def get_region_series(s):
    region, lon, lat = get_region(s["city"], s["ULICA"], s["street_number"])
    
    s["region"] = region
    s["lon"] = lon
    s["lat"] = lat
    return s

In [ ]:
get_region("Lodz", "Wolczanska", "143")

In [ ]:
tqdm.pandas(desc = "working working")

In [ ]:
df = df.progress_apply(lambda row: get_region_series(row), axis=1)

In [ ]:
df

In [ ]:
df.to_csv("pna_regions.csv", index=False)